In [30]:
!pip install --upgrade --quiet google-genai

In [31]:
from google.colab import userdata

GEMINI_API_KEY : str = userdata.get('GEMINI_API_KEY')

if GEMINI_API_KEY:
  print("GEMINI_API_KEY FOUND")
else:
  print("GEMINI_API_KEY NOT FOUND")

GEMINI_API_KEY FOUND


In [32]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

# Video Generation

Generated From Vivago.ai : https://vivago.ai/video-generation

**Prompt For video Generation** : An animated wizard stirs a bubbling, glowing potion in a cauldron inside a cozy, book-filled cottage. The camera zooms in on the cauldron as the potion turns bright purple, then quickly pans out as it bursts into a colorful explosion of sparkles and swirling light

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import os
file_path = '/content/drive/MyDrive/magician.mp4'
if os.path.exists(file_path):
  print(f"File '{file_path}' exists.")
else:
  print(f"File '{file_path}' does not exist in the specified location.")

File '/content/drive/MyDrive/magician.mp4' exists.


In [37]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

In [38]:
magician_video = upload_video(file_path)

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/lonja9gw7q39


In [39]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = magician_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)


In [40]:
from IPython.display import Markdown
display(Markdown(response.text))

```json
[
  {
    "timecode": "00:00",
    "caption": "A cartoon wizard with a blue robe and purple hat raises his hands over a cauldron in a room lined with bookshelves."
  },
  {
    "timecode": "00:01",
    "caption": "A cauldron filled with a bubbling, purple potion."
  },
  {
    "timecode": "00:02",
    "caption": "The purple potion in the cauldron spews upwards in a stream."
  },
   {
    "timecode": "00:03",
    "caption": "A close up shot of a bright pink liquid in a cauldron."
   },
  {
    "timecode": "00:04",
    "caption": "A close up shot of a bright pink liquid in a cauldron with streams flowing into it."
  }
]
```
